# 음원사이트 멜론 파싱

## #01. 패키지 준비

In [ ]:
import pandas as pd
from pandas import DataFrame
import requests
from bs4 import BeautifulSoup

In [ ]:
# 수집할 사이트 주소 설정
params = ['', 'day', 'week', 'month']

# dict 형식의 노래제목, 가수, 앨범, 순위를 담을 빈 list 생성
melon_chart = []

for p in params:
    # 수집할 컨텐츠가 있는 웹 페이지 주소
    url = f'https://www.melon.com/chart/{p}/index.htm'
    # print(url)


    # 브라우저 버전정보
    userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    # 접속 객체 생성
    # request는 직접 header 정보 갱신이 불가능해 session 생성
    session = requests.Session()

    # 접속 객체에 부가정보(header) 삽입
    session.headers.update({
        "Referer": "",
        "User-Agent": userAgent
    })

    # 생성한 접속 객체를 활용해 웹 페이지에 접속
    # 지정된 URL에 GET 요청을 보내고
    # 그에 대한 응답을 r 변수에 할당
    r = session.get(url)

    # 접속 실패 시 에러 코드와 에러 메시지 출력
    if r.status_code != 200:
        msg = "[%d Error] %s 에러 발생" % (r.status_code, r.reason)
        raise Exception(msg)

    # 인코딩 형식을 지정해 beautifulsoup 객체를 생성
    r.encoding = 'utf-8'

    # r 변수에 저장된 text 정보를 시각화, 확인
    # print(r.text)


    # r.text에서 HTML을 파싱하고,
    # 파싱된 HTML을 나타내는
    # BeautifulSoup 객체 soup를 생성
    soup = BeautifulSoup(r.text)
    # print(soup)



    # 추출할 음원의 수를 확인하기 위해 음악 이름을 나타내는
    # .ellipsis.rank01의 수를 확인
    melon_list = soup.select(".ellipsis.rank01")
    # len(melon_list)


    for i in range(len(melon_list)):
        # 노래 추출
        songEl = soup.select('.ellipsis.rank01 > span > a')
        song = songEl[i].text.strip()

        # 가수 추출
        singerEl = soup.select('.ellipsis.rank02 > a')
        singer = singerEl[i].text.strip()
        
        # 앨범 추출
        albumEl = soup.select('.ellipsis.rank03 > a')
        album = albumEl[i].text.strip()

        # 순위
        rank =+ (1+i)

        item = {"가수": singer, "노래 제목": song, "앨범": album, "순위" : rank}
        melon_chart.append(item)
melon_chart

In [ ]:
df = DataFrame(melon_chart)
# 순위를 우선해서 보여주기 위해 index로 설정
df.set_index('순위',inplace=True)
df.to_excel('확인용.xlsx')